# Hourly Demand Forecasting Model

**Business Problem**: Predict hourly bike demand to optimize fleet sizing, staffing, and bike redistribution.

**Approach**: 
1. Aggregate journey data to hourly counts
2. Engineer temporal features (hour, day of week, month, etc.)
3. Train ML models (XGBoost, Random Forest)
4. Evaluate and interpret results

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import xgboost as xgb
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

## 1. Load and Prepare Data

In [ ]:
# Load the cleaned dataset
df = pd.read_parquet("data/journeys_2019_2020_2021.parquet")

# Remove outliers (>24h journeys)
df = df[df['Duration'] < 1440].copy()

print(f"Total journeys: {len(df):,}")
print(f"Date range: {df['Start Date'].min()} to {df['Start Date'].max()}")

In [ ]:
# Aggregate to hourly demand (total journeys per hour)
df['hour_bucket'] = df['Start Date'].dt.floor('h')

hourly_demand = df.groupby('hour_bucket').size().reset_index(name='demand')
hourly_demand.columns = ['datetime', 'demand']

print(f"Hourly records: {len(hourly_demand):,}")
print(f"Average hourly demand: {hourly_demand['demand'].mean():.0f} journeys")
hourly_demand.head(10)

## 2. Feature Engineering

Create temporal features that capture cyclical patterns in bike demand.

In [ ]:
def create_features(df):
    """Create temporal features for demand forecasting."""
    df = df.copy()
    
    # Basic temporal features
    df['hour'] = df['datetime'].dt.hour
    df['day_of_week'] = df['datetime'].dt.dayofweek  # 0=Monday, 6=Sunday
    df['day_of_month'] = df['datetime'].dt.day
    df['month'] = df['datetime'].dt.month
    df['year'] = df['datetime'].dt.year
    df['week_of_year'] = df['datetime'].dt.isocalendar().week.astype(int)
    
    # Binary features
    df['is_weekend'] = (df['day_of_week'] >= 5).astype(int)
    df['is_rush_hour'] = df['hour'].isin([7, 8, 9, 17, 18, 19]).astype(int)
    
    # Cyclical encoding (captures circular nature of time)
    df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
    df['dow_sin'] = np.sin(2 * np.pi * df['day_of_week'] / 7)
    df['dow_cos'] = np.cos(2 * np.pi * df['day_of_week'] / 7)
    df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
    
    # Lag features (previous hour's demand)
    df['demand_lag_1h'] = df['demand'].shift(1)
    df['demand_lag_24h'] = df['demand'].shift(24)  # Same hour yesterday
    df['demand_lag_168h'] = df['demand'].shift(168)  # Same hour last week
    
    # Rolling averages
    df['demand_rolling_24h'] = df['demand'].shift(1).rolling(window=24).mean()
    df['demand_rolling_7d'] = df['demand'].shift(1).rolling(window=168).mean()
    
    return df

# Apply feature engineering
hourly_demand = create_features(hourly_demand)

# Drop rows with NaN (from lag features)
hourly_demand = hourly_demand.dropna()

print(f"Features created. Final dataset: {len(hourly_demand):,} rows")
print(f"\nFeatures: {list(hourly_demand.columns)}")

## 3. Train/Test Split

Use time-based split (not random) to respect temporal ordering - train on earlier data, test on later data.

In [ ]:
# Define features and target
feature_cols = [
    'hour', 'day_of_week', 'day_of_month', 'month', 'year', 'week_of_year',
    'is_weekend', 'is_rush_hour',
    'hour_sin', 'hour_cos', 'dow_sin', 'dow_cos', 'month_sin', 'month_cos',
    'demand_lag_1h', 'demand_lag_24h', 'demand_lag_168h',
    'demand_rolling_24h', 'demand_rolling_7d'
]

X = hourly_demand[feature_cols]
y = hourly_demand['demand']

# Time-based split: Train on 2019-2020, Test on 2021
train_mask = hourly_demand['year'] < 2021
X_train, X_test = X[train_mask], X[~train_mask]
y_train, y_test = y[train_mask], y[~train_mask]

print(f"Training set: {len(X_train):,} samples ({hourly_demand[train_mask]['datetime'].min().date()} to {hourly_demand[train_mask]['datetime'].max().date()})")
print(f"Test set: {len(X_test):,} samples ({hourly_demand[~train_mask]['datetime'].min().date()} to {hourly_demand[~train_mask]['datetime'].max().date()})")

## 4. Model Training

Train multiple models and compare performance.

In [ ]:
# Define models
models = {
    'Random Forest': RandomForestRegressor(
        n_estimators=100, 
        max_depth=15, 
        min_samples_split=5,
        n_jobs=-1, 
        random_state=42
    ),
    'XGBoost': xgb.XGBRegressor(
        n_estimators=100,
        max_depth=8,
        learning_rate=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        verbosity=0
    ),
    'Gradient Boosting': GradientBoostingRegressor(
        n_estimators=100,
        max_depth=8,
        learning_rate=0.1,
        random_state=42
    )
}

# Train and evaluate each model
results = {}

for name, model in models.items():
    print(f"Training {name}...")
    model.fit(X_train, y_train)
    
    # Predictions
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    
    # Metrics
    # MAPE calculation (handle zeros by adding small epsilon)
    train_mape = np.mean(np.abs((y_train - y_pred_train) / (y_train + 1e-8))) * 100
    test_mape = np.mean(np.abs((y_test - y_pred_test) / (y_test + 1e-8))) * 100
    
    results[name] = {
        'model': model,
        'train_mae': mean_absolute_error(y_train, y_pred_train),
        'test_mae': mean_absolute_error(y_test, y_pred_test),
        'train_rmse': np.sqrt(mean_squared_error(y_train, y_pred_train)),
        'test_rmse': np.sqrt(mean_squared_error(y_test, y_pred_test)),
        'train_r2': r2_score(y_train, y_pred_train),
        'test_r2': r2_score(y_test, y_pred_test),
        'train_mape': train_mape,
        'test_mape': test_mape,

        'predictions': y_pred_test
    }

    print("\nTraining complete!")
    print(f"  Test MAE: {results[name]['test_mae']:.1f} | Test MAPE: {results[name]['test_mape']:.1f}% | Test R²: {results[name]['test_r2']:.3f}")

## 5. Model Comparison

In [ ]:
# Create comparison dataframe
comparison_df = pd.DataFrame({
    name: {
        'Train MAE': f"{r['train_mae']:.1f}",
        'Test MAE': f"{r['test_mae']:.1f}",
        'Train RMSE': f"{r['train_rmse']:.1f}",
        'Test RMSE': f"{r['test_rmse']:.1f}",
        'Train MAPE': f"{r['train_mape']:.1f}%",
        'Test MAPE': f"{r['test_mape']:.1f}%",
        'Train R²': f"{r['train_r2']:.3f}",
        'Test R²': f"{r['test_r2']:.3f}"
    }
    for name, r in results.items()
}).T

print("=== Model Performance Comparison ===\n")
print(comparison_df.to_string())

# Identify best model
# Selection criteria: MAE (Mean Absolute Error)
# Rationale: 
# - MAE is operationally meaningful: "We're off by X bikes/hour on average"
# - Directly informs fleet buffer sizing decisions
# - MAPE is inflated by low-demand overnight hours (being off by 10 bikes at 3AM = 50% error)
# - High-demand accuracy matters more than low-demand percentage accuracy

best_model_name = min(results, key=lambda x: results[x]['test_mae'])
print(f"\n Best model (lowest Test MAE): {best_model_name}")
print(f"  - MAE chosen over MAPE: MAPE inflated by low-demand overnight hours where accuracy matters less")

In [ ]:
# Visualize model comparison
fig = make_subplots(rows=1, cols=2, subplot_titles=['Mean Absolute Error', 'R² Score'])

model_names = list(results.keys())
colors = ['#3498DB', '#E74C3C', '#27AE60']

# MAE comparison
fig.add_trace(
    go.Bar(
        x=model_names,
        y=[results[m]['train_mae'] for m in model_names],
        name='Train',
        marker_color='#3498DB',
        text=[f"{results[m]['train_mae']:.0f}" for m in model_names],
        textposition='outside'
    ),
    row=1, col=1
)
fig.add_trace(
    go.Bar(
        x=model_names,
        y=[results[m]['test_mae'] for m in model_names],
        name='Test',
        marker_color='#E74C3C',
        text=[f"{results[m]['test_mae']:.0f}" for m in model_names],
        textposition='outside'
    ),
    row=1, col=1
)

# R² comparison
fig.add_trace(
    go.Bar(
        x=model_names,
        y=[results[m]['train_r2'] for m in model_names],
        name='Train',
        marker_color='#3498DB',
        text=[f"{results[m]['train_r2']:.2f}" for m in model_names],
        textposition='outside',
        showlegend=False
    ),
    row=1, col=2
)
fig.add_trace(
    go.Bar(
        x=model_names,
        y=[results[m]['test_r2'] for m in model_names],
        name='Test',
        marker_color='#E74C3C',
        text=[f"{results[m]['test_r2']:.2f}" for m in model_names],
        textposition='outside',
        showlegend=False
    ),
    row=1, col=2
)

fig.update_layout(
    title='<b>Model Performance Comparison</b>',
    height=450,
    width=1000,
    barmode='group'
)
fig.update_yaxes(title_text='MAE (journeys/hour)', row=1, col=1)
fig.update_yaxes(title_text='R² Score', row=1, col=2)

fig.show()

## 6. Feature Importance

Understanding which factors drive bike demand.

In [ ]:
# Get feature importance from best model (XGBoost)
best_model = results[best_model_name]['model']

if hasattr(best_model, 'feature_importances_'):
    importance_df = pd.DataFrame({
        'feature': feature_cols,
        'importance': best_model.feature_importances_
    }).sort_values('importance', ascending=True)
    
    fig = go.Figure()
    fig.add_trace(go.Bar(
        y=importance_df['feature'],
        x=importance_df['importance'],
        orientation='h',
        marker_color='#2C3E50'
    ))
    
    fig.update_layout(
        title=f'<b>Feature Importance ({best_model_name})</b>',
        xaxis_title='Importance',
        height=600,
        width=800,
        margin=dict(l=200)
    )
    fig.show()
    
    print("\n=== Top 5 Most Important Features ===")
    for _, row in importance_df.tail(5).iloc[::-1].iterrows():
        print(f"  {row['feature']}: {row['importance']:.3f}")

## 7. Predictions vs Actual

Visualize how well the model predicts demand over time.

In [ ]:
# Create test set with predictions
test_df = hourly_demand[~train_mask].copy()
test_df['predicted'] = results[best_model_name]['predictions']

# Plot a 2-week sample for clarity
sample_start = '2021-06-01'
sample_end = '2021-06-14'
sample = test_df[(test_df['datetime'] >= sample_start) & (test_df['datetime'] <= sample_end)]

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=sample['datetime'],
    y=sample['demand'],
    mode='lines',
    name='Actual',
    line=dict(color='#3498DB', width=1.5)
))

fig.add_trace(go.Scatter(
    x=sample['datetime'],
    y=sample['predicted'],
    mode='lines',
    name='Predicted',
    line=dict(color='#E74C3C', width=1.5, dash='dot')
))

fig.update_layout(
    title=f'<b>Actual vs Predicted Hourly Demand</b><br><sup>Sample: {sample_start} to {sample_end}</sup>',
    xaxis_title='Date',
    yaxis_title='Hourly Demand (journeys)',
    height=500,
    width=1100,
    legend=dict(x=0.01, y=0.99),
    hovermode='x unified'
)

fig.show()

In [ ]:
# Scatter plot: Actual vs Predicted
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=y_test,
    y=results[best_model_name]['predictions'],
    mode='markers',
    marker=dict(color='#3498DB', opacity=0.3, size=4),
    name='Predictions'
))

# Perfect prediction line
max_val = max(y_test.max(), results[best_model_name]['predictions'].max())
fig.add_trace(go.Scatter(
    x=[0, max_val],
    y=[0, max_val],
    mode='lines',
    line=dict(color='#E74C3C', dash='dash'),
    name='Perfect Prediction'
))

fig.update_layout(
    title=f'<b>Actual vs Predicted ({best_model_name})</b><br><sup>R² = {results[best_model_name]["test_r2"]:.3f}</sup>',
    xaxis_title='Actual Demand',
    yaxis_title='Predicted Demand',
    height=500,
    width=600
)

fig.show()

## 8. Business Application: Demand by Hour of Week

Show predicted demand patterns the client can use for operational planning.

In [ ]:
# Create heatmap of average demand by day of week and hour
heatmap_data = test_df.groupby(['day_of_week', 'hour'])['demand'].mean().unstack()

# Rename for clarity
day_names = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
heatmap_data.index = day_names

fig = go.Figure(data=go.Heatmap(
    z=heatmap_data.values,
    x=[f'{h:02d}:00' for h in range(24)],
    y=day_names,
    colorscale='RdYlGn_r',
    colorbar=dict(title='Avg Demand')
))

fig.update_layout(
    title='<b>Average Hourly Demand by Day of Week</b><br><sup>Use for staffing and bike redistribution planning</sup>',
    xaxis_title='Hour of Day',
    yaxis_title='Day of Week',
    height=400,
    width=1000
)

fig.show()

print("\n=== Key Insights for Operations ===")
peak_hour = test_df.groupby('hour')['demand'].mean().idxmax()
peak_day = test_df.groupby('day_of_week')['demand'].mean().idxmax()
print(f"• Peak hour: {peak_hour}:00")
print(f"• Peak day: {day_names[peak_day]}")
print(f"• Average hourly demand: {test_df['demand'].mean():.0f} journeys")
print(f"• Peak hourly demand: {test_df['demand'].max():,.0f} journeys")

## 9. Summary & Business Recommendations

### Model Performance
- The model can predict hourly demand with high accuracy, enabling proactive operational planning.

### Business Applications
1. **Fleet Sizing**: Use predicted peak demand to determine required bike inventory
2. **Staff Scheduling**: Align maintenance/redistribution crews with demand valleys (night, early morning)
3. **Dynamic Pricing**: Increase prices during predicted high-demand periods to manage demand
4. **Bike Redistribution**: Pre-position bikes before predicted morning/evening rushes

### Next Steps
- Add external data (weather, events, holidays) to improve predictions
- Build station-level forecasting for granular redistribution planning
- Implement real-time forecasting pipeline for production use